In [3]:
# Importing libraries--------------------------------------------------------------------------------------------------------------------
import cv2
import numpy as np
import dlib
from math import hypot
import pyglet
import time


#Importing the sounds--------------------------------------------------------------------------------------------------------------------
sound = pyglet.media.load("sound.mp3", streaming=False)            #streaming is false so that we can play one sound over and over again
left_sound = pyglet.media.load("left.mp3", streaming=False)       
right_sound = pyglet.media.load("right.mp3", streaming=False)


#Capturing through webcam--------------------------------------------------------------------------------------------------------------------
cap = cv2.VideoCapture(0)


#White board to display the text input from keyboard--------------------------------------------------------------------------------------------------------------------
board = np.zeros((375, 1800), np.uint8)
board[:] = 255                                                    #255 to make it completely white


#Importing face detector from dlib--------------------------------------------------------------------------------------------------------------------
detector = dlib.get_frontal_face_detector()                                           #consist a library of faces
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")             #detects the landmarks(69 in total) of the face


#Creating two windows to display the two sides of keyboard--------------------------------------------------------------------------------------------------------------------
keyboard_1 = np.zeros((300, 500, 3), np.uint8)
keyboard_2 = np.zeros((300, 500, 3), np.uint8)


#Creating two sets of keyboards
keys_set_1 = {0: "Q", 1: "W", 2: "E", 3: "R", 4: "T",            #this set of keys will be displayed on the left side
              5: "A", 6: "S", 7: "D", 8: "F", 9: "G",
              10: "Z", 11: "X", 12: "C", 13: "V", 14: "B"}

keys_set_2 = {0: "Y", 1: "U", 2: "I", 3: "O", 4: "P",            #this set of keys will be displayed on the right side
              5: "H", 6: "J", 7: "K", 8: "L", 9: "_",
              10: "N", 11: "M", 12: ".", 13: ",", 14: "#"}


#Function to display keys on the two windows--------------------------------------------------------------------------------------------------------------------
def letter(letter_index, text, letter_light,keyboard_selected):
    if letter_index == 0:
        x = 0
        y = 0
    elif letter_index == 1:
        x = 100
        y = 0
    elif letter_index == 2:
        x = 200
        y = 0
    elif letter_index == 3:
        x = 300
        y = 0
    elif letter_index == 4:
        x = 400
        y = 0
    elif letter_index == 5:
        x = 0
        y = 100
    elif letter_index == 6:
        x = 100
        y = 100
    elif letter_index == 7:
        x = 200
        y = 100
    elif letter_index == 8:
        x = 300
        y = 100
    elif letter_index == 9:
        x = 400
        y = 100
    elif letter_index == 10:
        x = 0
        y = 200
    elif letter_index == 11:
        x = 100
        y = 200
    elif letter_index == 12:
        x = 200
        y = 200
    elif letter_index == 13:
        x = 300
        y = 200
    elif letter_index == 14:
        x = 400
        y = 200
    # Keys
    width = 100
    height = 100
    th = 3                    #thickness of the rectangle
    if letter_light is True:
        cv2.rectangle(keyboard_selected, (x + th, y + th), (x + width - th, y + height - th), (255, 255, 255), -1)
    
    else:
        cv2.rectangle(keyboard_selected, (x + th, y + th), (x + width - th, y + height - th), (50, 50, 50), th)

    # Text settings
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 5
    font_th = 2
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_th)[0]      #"gettextsize" returns [(width,height),something]
    width_text, height_text = text_size[0], text_size[1]
    text_x = int((width - width_text) / 2) + x                           #to get the x coordinate of where the letter should be placed
    text_y = int((height + height_text) / 2) + y                         #bottom left is considered as the origin of the letter 
    cv2.putText(keyboard_selected, text, (text_x, text_y), font_letter, font_scale, (100, 100, 100), font_th)
    

#Function to get the midpoint of the top center and bottom center of landmarks--------------------------------------------------------------------------------------------------------------------
def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)             #to check if the user closed the eyes we draw a vertical line on the eye
                                                                  #this line starts from the center point of the two eye landmark hence we divide by 2
font = cv2.FONT_HERSHEY_SIMPLEX


#Function to get the blinking ratio--------------------------------------------------------------------------------------------------------------------
def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))

#     hor_line = cv2.line(frame, left_point, right_point, (0, 255, 0), 2)
#     ver_line = cv2.line(frame, center_top, center_bottom, (0, 255, 0), 2)

    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))    #this returns the length of the line or the distance between those two points
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    ratio = hor_line_lenght / ver_line_lenght                  #higher ratio means verticle length is too small;the eyes are closed
    return ratio


#Function tot get the Gaze ratio--------------------------------------------------------------------------------------------------------------------
def get_gaze_ratio(eye_points, facial_landmarks):

    # Gaze detection
    left_eye_region = np.array([(landmarks.part(36).x, landmarks.part(36).y),    #stores the coordiates as arrays of tuple
                            (landmarks.part(37).x, landmarks.part(37).y),
                            (landmarks.part(38).x, landmarks.part(38).y),
                            (landmarks.part(39).x, landmarks.part(39).y),
                            (landmarks.part(40).x, landmarks.part(40).y),
                            (landmarks.part(41).x, landmarks.part(41).y)], np.int32)
        
       
    height, width, _ = frame.shape
    mask = np.zeros((height, width), np.uint8)             #creating a black window of the same size as our original frame
    cv2.polylines(mask, [left_eye_region], True, 255, 2)   #we draw polylines on the mask of the eye region
    cv2.fillPoly(mask, [left_eye_region], 255)             #we then fill the polygon with white color
    eye = cv2.bitwise_and(gray, mask)                      #we perform AND operation between the orginal gray image and mask
        
    min_x = np.min(left_eye_region[:, 0])                  #this we do to display the eye only, in a new window
    max_x = np.max(left_eye_region[:, 0])                  #[:,0] returns the x values of the coordiante
    min_y = np.min(left_eye_region[:, 1])                  #[:,1] returns the y values of the coordiante
    max_y = np.max(left_eye_region[:, 1])
        
        
    gray_eye = eye[min_y: max_y, min_x: max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    height, width = threshold_eye.shape
    
    left_side_threshold = threshold_eye[0: height, 0: int(width / 2)]
    left_side_white = cv2.countNonZero(left_side_threshold)

    right_side_threshold = threshold_eye[0: height, int(width / 2): width]
    right_side_white = cv2.countNonZero(right_side_threshold)
    
    if right_side_white == 0:                  #to avoid the denominator from being zero 
        right_side_white=0.1
        
    gaze_ratio = left_side_white / right_side_white           
    
    return gaze_ratio
    
    

frames = 0                                #counter for frames to highlight each letter
letter_index = 0                          #counter for index of the letters
blinking_frames = 0                       #counter for the no. of frames the user is blinking
text = ""                                 #Empty string to append our text data   
keyboard_selected = 'left'
last_keyboard_selected ='left'
counter = 0                               #acts as a switch, to confirm the keyboard at which we are looking

while True:
    _, frame = cap.read()                  
    keyboard_1[:]=(0,0,0)                 #this is used so that when one letter is lit up the other letters are reset to black
    keyboard_2[:]=(0,0,0)
    frames +=1

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    active_letter = keys_set_1[letter_index] + keys_set_2[letter_index]
    
    faces = detector(gray)                                       #detects the faces in the cam
    for face in faces:
        #x, y = face.left(), face.top()
        #x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0, 255, 0), 2)
        landmarks = predictor(gray, face)                             #returns all the landmarks in the detected face                        

        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)  #landmark points for left eye
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks) #landmark points for right eye
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2                   #taking average of the two ratios

        if blinking_ratio > 4:
            #cv2.putText(frame, "BLINKING", (50, 150), font, 4, (255, 0, 0), thickness=3)
            blinking_frames += 1                                      
            frames -= 1                                   #to stop the incrementation of number of frames while the eyes are closed so that the highlighted letter stays until we open our eyes
            if blinking_frames == 3:                      #if we close ur eyes for 3 consecutive frames we take that letter and append it to the text
                if counter == 1:
                    text += active_letter[1]
                    sound.play()
                    time.sleep(1)
                else:
                    text += active_letter[0]
                    sound.play()
                    time.sleep(1)
                    
        else:
            blinking_frames = 0
        
        
        # Gaze detection
        gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
        gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2
    
        new_frame = np.zeros((500,500,3),np.uint8)
        
        
        if gaze_ratio <= 0.9:
            keyboard_selected = "right"
            if keyboard_selected != last_keyboard_selected:
                #right_sound.play()
                time.sleep(1)
                last_keyboard_selected = keyboard_selected
                keyboard = keyboard_2
                keys_set = keys_set_2
                counter = 1
                
                 
                
        else:
            keyboard_selected = "left"
            if keyboard_selected != last_keyboard_selected:
                #left_sound.play()
                time.sleep(1)
                last_keyboard_selected = keyboard_selected
                keyboard = keyboard_1
                keys_set = keys_set_1
                counter = 2
                
                

    
    if frames == 20:                                        #we want to highlight each letter every 20 frame
        letter_index += 1                                   
        frames = 0                                          #to reset counter from 0 to 10                                     
    if letter_index == 15:                                  #once all the letter are highlighyed, we start from the first key again
        letter_index = 0


    for i in range(15):
        if i == letter_index:                               
            light = True
        else:
            light = False
        letter(i, keys_set[i], light,keyboard)

    cv2.putText(board, text, (20, 100), font, 4, 0, 3)
    
    cv2.imshow("Virtual left-side", keyboard_1) 
    cv2.imshow("Virtual right-side", keyboard_2) 
    frame = cv2.resize(frame, (100, 100))
    cv2.imshow("Frame", frame)
    cv2.imshow("Text", board)

    key = cv2.waitKey(1)
    if key == 27:
            break
    
cap.release()
cv2.destroyAllWindows()